In [150]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import time
import json
from typing import Literal
from travel_map import TravelMap
import requests

_ : bool = load_dotenv(find_dotenv())

MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")


client:OpenAI = OpenAI()

In [171]:
import uuid


def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))


customMap = TravelMap()

def update_travel_map(**kwargs) -> Literal['updated']:
    mainLocation=dict(kwargs).get("mainLocation")
    suggestLocations=kwargs.get("suggestLocations")
    print(mainLocation)
    print(suggestLocations)
    if mainLocation != None and suggestLocations != None: 
        suggestLocations.insert(0,mainLocation)  
   
    customMap.updateMap(mainLocation, suggestLocations)
    return 'updated'


# update_travel_map(mainLocation={'test':'test'},suggestLocations=[])

def getLocations(**kwargs):
    searchText=dict(kwargs).get("searchText")
    type=kwargs.get("type")
    country=kwargs.get("country")
    print(searchText)
    print(type)
    print(country)
    suggestResponse = requests.get(f'https://api.mapbox.com/geocoding/v5/mapbox.places/{searchText}.json?access_token={MAPBOX_TOKEN}&type={type}&country={country}')
    # suggestResponse = requests.get(f'https://api.mapbox.com/search/searchbox/v1/suggest?q={searchText}&session_token={uuid.uuid4()}&access_token={MAPBOX_TOKEN}&limit=5&country={country}')

    print(suggestResponse.json())
    return str(suggestResponse.json()['features'])

available_functions = {
    "update_travel_map":update_travel_map ,
    "getLocations":getLocations,
} 

In [172]:
# getLocations(searchText='dolmen',country='pk')

dolmen
None
pk
{'type': 'FeatureCollection', 'query': ['dolmen'], 'features': [{'id': 'poi.549755824855', 'type': 'Feature', 'place_type': ['poi'], 'relevance': 1, 'properties': {'foursquare': '4eb7057d1081376a2c5b5f05', 'landmark': True, 'wikidata': 'Q15983397', 'address': 'HC-3, Block-4, Marine Drive, Clifton', 'category': 'clothing, accessories, apparel, shop, clothes'}, 'text': 'Dolmen Mall Clifton', 'place_name': 'Dolmen Mall Clifton, HC-3, Block-4, Marine Drive, Clifton, Karachi, Sindh, Pakistan', 'center': [67.0295595, 24.8022515], 'geometry': {'coordinates': [67.0295595, 24.8022515], 'type': 'Point'}, 'context': [{'id': 'locality.37710516', 'mapbox_id': 'dXJuOm1ieHBsYzpBajlxdEE', 'text': 'Civil Line'}, {'id': 'place.67764', 'mapbox_id': 'dXJuOm1ieHBsYzpBUWkw', 'wikidata': 'Q8660', 'text': 'Karachi'}, {'id': 'district.722612', 'mapbox_id': 'dXJuOm1ieHBsYzpDd2Ew', 'wikidata': 'Q6367783', 'text': 'کراچی جنوبی'}, {'id': 'region.42164', 'mapbox_id': 'dXJuOm1ieHBsYzpwTFE', 'wikidata'

"[{'id': 'poi.549755824855', 'type': 'Feature', 'place_type': ['poi'], 'relevance': 1, 'properties': {'foursquare': '4eb7057d1081376a2c5b5f05', 'landmark': True, 'wikidata': 'Q15983397', 'address': 'HC-3, Block-4, Marine Drive, Clifton', 'category': 'clothing, accessories, apparel, shop, clothes'}, 'text': 'Dolmen Mall Clifton', 'place_name': 'Dolmen Mall Clifton, HC-3, Block-4, Marine Drive, Clifton, Karachi, Sindh, Pakistan', 'center': [67.0295595, 24.8022515], 'geometry': {'coordinates': [67.0295595, 24.8022515], 'type': 'Point'}, 'context': [{'id': 'locality.37710516', 'mapbox_id': 'dXJuOm1ieHBsYzpBajlxdEE', 'text': 'Civil Line'}, {'id': 'place.67764', 'mapbox_id': 'dXJuOm1ieHBsYzpBUWkw', 'wikidata': 'Q8660', 'text': 'Karachi'}, {'id': 'district.722612', 'mapbox_id': 'dXJuOm1ieHBsYzpDd2Ew', 'wikidata': 'Q6367783', 'text': 'کراچی جنوبی'}, {'id': 'region.42164', 'mapbox_id': 'dXJuOm1ieHBsYzpwTFE', 'wikidata': 'Q37211', 'short_code': 'PK-SD', 'text': 'Sindh'}, {'id': 'country.8884', '

In [153]:
# creating assistant

assistant = client.beta.assistants.update(
    assistant_id="asst_HnH3lC8ggTXM2pUFwnZhVvjF",
    name="Travel Assistant",
    instructions="""
        -you are a experienced travel assiatant, 
        -your job is to suggest the user best places you can also use map 
        -always suggest places near to main location if not specified.
        -map is mandatory
    """,
            
    model="gpt-3.5-turbo-1106",
    # model="gpt-4-turbo-preview",

    tools=[
        {"type": "code_interpreter"},
        {
            "type":'function',
            "function":{
                "name":"update_travel_map",
                "description":"""
                    -use this function for updating the locaiton on the map, this function uses the mapbox with plotly python
                    
                    -function has two parameters in delimiter
                     `mainLocation` : this  dictionary is  the center of the map (main location from user preference)
                     `suggestLocations` : this is the list of dictionaries(any extra suggest locations)

                    -keys of location object is in delimiter
                     `name`, `lat`, `lon`

                    -always ensure and check pass these two arguments never call functions without arguments
                     function return `updated` which means map is updated.
                    """,
               "parameters": {
                    "type": "object",
                    "properties": {
                    "mainLocation": {"type": "object", "description": "this is the single object"},
                    "suggestLocations": {"type": "object", "description": "this is the list of objects"},
                },
                "required": ["mainLocation",'suggestLocations']
                }

            }
        },
        {
            'type':'function',
            'function':{
                "name":'getLocations',
                'description':"""
                    you can use this to get the locations data for `update_travel_map` function like name,latitute,longitute etc
                    this function uses the mapbox.places of geocoding api 'https://docs.mapbox.com/api/search/geocoding/' 
                """,
                "parameters":{
                    "type":"object",
                    "properties":{
                        'searchText':{'type':'string','description':'location place text for search query text, text without country name '},
                        'type':{'type':'string','description':"""
                                types of location
                                country, region, postcode, district, place, locality, neighborhood, address
                                for multipe provide in comma seprated
                                """
                            },
                        'country':{'type':'string','description':'country of the search location, use ISO 3166-1 alpha-2 code for this'}
                    },
                    'required':["searchText",'type','country']
                }
            }
        }
    ],
    #  tool_choice={"type": "function", "function": {"name": "update_travel_map"}}
)

# assistant = dict(id='asst_HnH3lC8ggTXM2pUFwnZhVvjF')

In [154]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_LT8bQ6XKyXPcvwrXwMdz3qiw', created_at=1707236063, metadata={}, object='thread')


In [155]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="where is Dilkusha Forum building in karachi, pakistan?"
)

In [156]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [157]:


  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        # print(runStatus.status ,'.....')
        # print("Status: ", "requires_action")
        # show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            # print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
             
                if function_name in available_functions:
    
                    function_to_call = available_functions[function_name]
                    response = function_to_call(**function_args)
                  
                    tool_outputs.append({
                              "tool_call_id": toolcall.id,
                              "output": response
                    })
                    
            # print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        # print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")

        # print('showing map')
        customMap.showMap()
        break  # Exit the loop after processing the completed run
        

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break



in_progress .....
Run is queued. Waiting...
requires_action .....
Dilkusha Forum building
building
PK
{'type': 'FeatureCollection', 'query': ['dilkusha', 'forum', 'building'], 'features': [], 'attribution': 'NOTICE: © 2024 Mapbox and its suppliers. All rights reserved. Use of this data is subject to the Mapbox Terms of Service (https://www.mapbox.com/about/maps/). This response and the information it contains may not be retained. POI(s) provided by Foursquare.'}
in_progress .....
Run is queued. Waiting...
completed .....
Assistant: I couldn't find the exact location for the Dilkusha Forum building in Karachi, Pakistan. Is there anything else I can help you with?

User: where is Dilkusha Forum building in karachi, pakistan?



In [158]:
# import plotly.graph_objects as go 
# from travel_map import TravelMap
# suggestLocations:[dict] = [
                           
#     {
#         "name":"Perris",
#         'lat':33.7825194,
#         'lon':-117.22864779999999,
#     }
# ]

# mainLocation:dict = {
#     "name":"Marysville",
#     'lat':48.0517637,
#     'lon':-122.1770818,
# }

# cmap = TravelMap()
# cmap.updateMap(mainLocation, suggestLocations)
# cmap.showMap()
# lat = [sl['lat'] for sl in suggestLocations if "lat" in sl]
# lon = [sl['lon'] for sl in suggestLocations if "lon" in sl]
# name = [sl['name'] for sl in suggestLocations if 'name' in sl]

# print(lat)
# print(lon)
# print(name)

# fig = go.Figure(go.Scattermapbox(
#     # df=mapData,
#     mode = "markers",
#     lon = lon, lat = lat,
#     hovertext=name,
#     marker=dict(
#         color='red',
#         size=10,
    
#     ),
#     textposition = "bottom right"))

# fig.update_layout(
#     mapbox = {
#         'zoom':15,
#         'accesstoken': MAPBOX_TOKEN,
#         'style': "outdoors",
#         "center":{'lat': mainLocation['lat'], 'lon': mainLocation['lon']}
        
#         },
#     showlegend = False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.update_traces(marker_color='red', selector=dict(type='scattermapbox'))
# # print(fig.show)
# fig.show()